Feature engineering

In [78]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import geopy
import collections
import geopandas as gpd
from scipy.spatial.distance import cdist

In [79]:
# read in the scraped and pre-processed past listing data
files_dire = '../data/curated/'
df = pd.read_csv(f'{files_dire}/processed_listing.csv').iloc[: , 1:]
# sort the dataframe so the same property are followed with sorted list date
df = df.sort_values(['address', 'list_date'], ascending=[True, False]).reset_index().iloc[:,1:]
df

,bed,bath,car,type,address,suburb,code,url,loc_address,lat,lon,weekly_rent,list_date
0,3,1,2,AUF,". GLYNDON ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glyndon Road, Camberwell, Melbourne, City of B...",-37.836230,145.079003,525.0,2022-08-01
1,3,1,2,AUF,". GLYNDON ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glyndon Road, Camberwell, Melbourne, City of B...",-37.836230,145.079003,349.0,2022-07-01
2,3,1,2,AUF,". GLYNDON ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glyndon Road, Camberwell, Melbourne, City of B...",-37.836230,145.079003,375.0,2022-04-01
3,3,1,2,House,".97 HUNTINGDALE ROAD, ASHWOOD",Ashwood,3147,https://www.oldlistings.com.au/real-estate/VIC...,"Huntingdale Road, Ashwood, Melbourne, City of ...",-37.864211,145.112405,430.0,2021-02-01
4,3,1,2,House,".97 HUNTINGDALE ROAD, ASHWOOD",Ashwood,3147,https://www.oldlistings.com.au/real-estate/VIC...,"Huntingdale Road, Ashwood, Melbourne, City of ...",-37.864211,145.112405,450.0,2021-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
417339,1,1,1,NaN,"YG02/125 TURNER STREET, ABBOTSFORD",Abbotsford,3067,https://www.oldlistings.com.au/real-estate/VIC...,"125 Turner St, Abbotsford VIC 3067, Australia",-37.799324,145.000522,425.0,2022-06-01
417340,1,1,1,NaN,"YG02/125 TURNER STREET, ABBOTSFORD",Abbotsford,3067,https://www.oldlistings.com.au/real-estate/VIC...,"125 Turner St, Abbotsford VIC 3067, Australia",-37.799324,145.000522,390.0,2015-11-01
417341,1,1,1,unknown,"YG09/125 TURNER STREET, ABBOTSFORD",Abbotsford,3067,https://www.oldlistings.com.au/real-estate/VIC...,"125 Turner St, Abbotsford VIC 3067, Australia",-37.799324,145.000522,395.0,2017-09-01
417342,1,1,1,unknown,"YG09/125 TURNER STREET, ABBOTSFORD",Abbotsford,3067,https://www.oldlistings.com.au/real-estate/VIC...,"125 Turner St, Abbotsford VIC 3067, Australia",-37.799324,145.000522,390.0,2017-09-01


In [80]:
df[df['suburb']=='Camberwell']

,bed,bath,car,type,address,suburb,code,url,loc_address,lat,lon,weekly_rent,list_date
0,3,1,2,AUF,". GLYNDON ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glyndon Road, Camberwell, Melbourne, City of B...",-37.836230,145.079003,525.0,2022-08-01
1,3,1,2,AUF,". GLYNDON ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glyndon Road, Camberwell, Melbourne, City of B...",-37.836230,145.079003,349.0,2022-07-01
2,3,1,2,AUF,". GLYNDON ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glyndon Road, Camberwell, Melbourne, City of B...",-37.836230,145.079003,375.0,2022-04-01
4215,4,2,3,NaN,"1 NEVIS STREET, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Nevis Street, Camberwell, Melbourne, City of B...",-37.843101,145.080403,850.0,2022-08-01
4216,4,2,3,NaN,"1 NEVIS STREET, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Nevis Street, Camberwell, Melbourne, City of B...",-37.843101,145.080403,750.0,2022-08-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
414790,2,2,2,NaN,"G19/828 BURKE ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"g19/828 Burke Rd, Camberwell VIC 3124, Australia",-37.821923,145.058748,580.0,2019-08-01
414917,2,1,1,AUF,"G45, 347 CAMBERWELL ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"g45/347 Camberwell Rd, Camberwell VIC 3124, Au...",-37.834044,145.060130,500.0,2021-02-01
414918,2,1,1,AUF,"G45, 347 CAMBERWELL ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"g45/347 Camberwell Rd, Camberwell VIC 3124, Au...",-37.834044,145.060130,520.0,2021-02-01
416864,3,2,2,NaN,"UNIT 301/828 BURKE ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"301/828 Burke Rd, Camberwell VIC 3124, Australia",-37.821923,145.058748,750.0,2022-03-01


In [81]:
# original size of the dataframe
len(df)

417344

In [82]:
# convert the date to a date format as it cannot be auto recognised by Pandas
df['list_date'] = df['list_date'].apply(pd.to_datetime)
# remove the data that has not been listed in 2021 or 2022
addresses = df.loc[df['list_date'].dt.year == 2022]['address'].tolist()
addresses = df.loc[df['list_date'].dt.year == 2021]['address'].tolist() + addresses
addresses = list(set(addresses))

In [83]:
len(addresses)

69669

In [84]:
# size of the reduced dataframe
df = df.loc[df['address'].isin(addresses)]
len(df)

262192

In [85]:
# store the number of entries for the loop below
n_rows = len(df)

In [86]:
# print the number of unique properties to have a rough idea
len(addresses)

69669

In [87]:
# copy the first row of the dataframe to df2
# df2 will only store each property per row to replace the prvious dataframe
df2 = df.iloc[0:1,:]
# listing history will be stored in a new column as a list with the format of [date difference in years, price difference]
df2 = df2.assign(list_history = '')
df2.head()

,bed,bath,car,type,address,suburb,code,url,loc_address,lat,lon,weekly_rent,list_date,list_history
0,3,1,2,AUF,". GLYNDON ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glyndon Road, Camberwell, Melbourne, City of B...",-37.83623,145.079003,525.0,2022-08-01,


In [88]:
# initialise for loop variables
n_property = 0
past_listing = []
list_date = df.iloc[0]['list_date']
list_price = df.iloc[0]['weekly_rent']

In [89]:
# run through all entries in the initial dataframe
for i in tqdm(range(1, n_rows)):
    # if the following row mataches with the previous one
    # calculate the date and price difference
    if df.iloc[i]['address'] == df2.iloc[n_property]['address']:
        days = (df.iloc[i]['list_date'] - list_date)/ np.timedelta64(1, 'Y')
        list_date = df.iloc[i]['list_date']
        price = (list_price - df.iloc[i]['weekly_rent']) / df.iloc[i]['weekly_rent']
        list_price = df.iloc[i]['weekly_rent']
        if days != 0:
            temp = [days, price]
            past_listing.append(temp)
    # if the address is different (different property)
    # the past listings are stored
    else:
        df2.at[n_property, 'list_history'] = past_listing
        n_property += 1
        df2.loc[n_property] = df.iloc[i]
        past_listing = []
        list_date = df.iloc[i]['list_date']
        list_price = df.iloc[i]['weekly_rent']
# manually store the last record
df2.at[n_property, 'list_history'] = past_listing

100%|██████████| 262191/262191 [05:59<00:00, 729.81it/s]


In [90]:
# a new column to store the number of times that the property has been listed for lease
df2['list_count'] = df2['list_history'].str.len()

In [91]:
# rename the column for legibility
df2 = df2.rename(columns={'code': 'postcode'}, errors="coerce")
# this file is stored to share with other group members
# df2.to_csv('../data/curated/processed_listing_combined.csv')

In [92]:
df2.head(20)

,bed,bath,car,type,address,suburb,postcode,url,loc_address,lat,lon,weekly_rent,list_date,list_history,list_count
0,3,1,2,AUF,". GLYNDON ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glyndon Road, Camberwell, Melbourne, City of B...",-37.836230,145.079003,525.0,2022-08-01,"[[-0.08487511721664373, 0.504297994269341], [-...",2
1,3,1,2,House,".97 HUNTINGDALE ROAD, ASHWOOD",Ashwood,3147,https://www.oldlistings.com.au/real-estate/VIC...,"Huntingdale Road, Ashwood, Melbourne, City of ...",-37.864211,145.112405,430.0,2021-02-01,[],0
2,3,1,1,House,"0 LANCASTER DRIVE, MANGALORE",Mangalore,3663,https://www.oldlistings.com.au/real-estate/VIC...,"Lancaster Drive, Mangalore, Shire of Strathbog...",-36.891826,145.183443,300.0,2022-02-01,"[[-8.000164274420419, 0.2], [-1.25122350219374...",2
3,2,2,1,NaN,"01/260 BURWOOD HIGHWAY, BURWOOD",Burwood,3125,https://www.oldlistings.com.au/real-estate/VIC...,"01/260 Burwood Hwy, Burwood VIC 3125, Australia",-37.850622,145.117896,480.0,2022-04-01,"[[-0.08487511721664373, 0.0]]",1
4,1,1,1,AUF,"013/80 CHELTENHAM ROAD, DANDENONG",Dandenong,3175,https://www.oldlistings.com.au/real-estate/VIC...,"013/80 Cheltenham Rd, Dandenong VIC 3175, Aust...",-37.990043,145.203630,246.0,2022-07-01,"[[-0.24914953763595418, 0.0]]",1
5,2,1,1,Townhouse,"02/844 PASCOE VALE ROAD, GLENROY",Glenroy,3046,https://www.oldlistings.com.au/real-estate/VIC...,"02/844 Pascoe Vale Rd, Glenroy VIC 3046, Austr...",-37.702619,144.915701,350.0,2021-11-01,"[[-4.084957254426853, -0.027777777777777776]]",1
6,1,1,1,NaN,"02/85 HUTTON STREET, THORNBURY",Thornbury,3071,https://www.oldlistings.com.au/real-estate/VIC...,"02/85 Hutton St, Thornbury VIC 3071, Australia",-37.755393,144.993613,380.0,2021-12-01,"[[-1.3333607124034033, -0.11627906976744186]]",1
7,2,1,1,AUF,"0206, 8 QUEENS AVENUE, HAWTHORN",Hawthorn,3122,https://www.oldlistings.com.au/real-estate/VIC...,"0206/8 Queens Ave, Hawthorn VIC 3122, Australia",-37.822953,145.043206,390.0,2021-12-01,"[[-0.5010369822788969, -0.04878048780487805]]",1
8,2,1,1,NaN,"03/85 HUTTON STREET, THORNBURY",Thornbury,3071,https://www.oldlistings.com.au/real-estate/VIC...,"03/85 Hutton St, Thornbury VIC 3071, Australia",-37.755393,144.993613,395.0,2022-02-01,"[[-0.16975023443328746, 0.0], [-1.916534904891...",2
9,3,1,1,AUF,"04/92 CADE WAY, PARKVILLE",Parkville,3052,https://www.oldlistings.com.au/real-estate/VIC...,"04/92 Cade Way, Parkville VIC 3052, Australia",-37.778226,144.938826,425.0,2021-12-01,"[[-0.5010369822788969, -0.1414141414141414], [...",2


In [93]:
# compare with the previous output to make sure we have roughly the expected amount of output
len(df2)

69669

In [94]:
# pivot to the number of past listings
col_list = (df2['list_count'].values.tolist())
col_list.sort()
counter = collections.Counter(col_list)
print(counter)
# it means there are 3xxxx properties that have been listed once in the past, etc

Counter({1: 25502, 2: 16147, 3: 10237, 4: 6614, 5: 4006, 6: 2668, 7: 1611, 8: 914, 0: 615, 9: 572, 10: 329, 11: 189, 12: 89, 13: 52, 14: 38, 15: 17, 16: 13, 17: 10, 18: 8, 20: 8, 19: 6, 21: 5, 26: 4, 34: 3, 27: 2, 29: 2, 30: 2, 36: 2, 23: 1, 24: 1, 31: 1, 42: 1})


### Embed SA2 area information to the existing dataframe

In [95]:
# read in the postcode information
postcode_df = pd.read_csv('../data/raw/abs/australian_postcodes.csv')
# only retain information that is relavent for faster running time
postcode_df = postcode_df.loc[postcode_df['state'] == 'VIC']
# rename columns to match dataframe so it can perform a 'vlookup' properly
postcode_df['locality'] = postcode_df['locality'].str.title()
postcode_df = postcode_df[['postcode', 'locality', 'SA2_NAME_2016', 'SA2_MAINCODE_2016', 'lgaregion']]
postcode_df = postcode_df.rename(columns={'locality': 'suburb'}, errors="raise")
postcode_df.head()

,postcode,suburb,SA2_NAME_2016,SA2_MAINCODE_2016,lgaregion
6151,3000,Melbourne,Melbourne,206041122.0,Melbourne
6152,3001,Melbourne,Melbourne,206041122.0,Moonee Valley
6153,3002,East Melbourne,East Melbourne,206041119.0,Yarra
6154,3003,West Melbourne,West Melbourne,206041127.0,Melbourne
6155,3004,Melbourne,Southbank,206041126.0,Yarra


In [96]:
# df3 will be the final output dataframe
# the property informations are merged with its SA2 information by matching both postcode and suburb name
df3 = pd.merge(df2, postcode_df, on=['postcode', 'suburb'])
df3['SA2_MAINCODE_2016'] = df3['SA2_MAINCODE_2016'].astype(int)
df3 = df3.rename(columns={'SA2_MAINCODE_2016': 'SA2'}, errors="raise")
df3.head()

,bed,bath,car,type,address,suburb,postcode,url,loc_address,lat,lon,weekly_rent,list_date,list_history,list_count,SA2_NAME_2016,SA2,lgaregion
0,3,1,2,AUF,". GLYNDON ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glyndon Road, Camberwell, Melbourne, City of B...",-37.836230,145.079003,525.0,2022-08-01,"[[-0.08487511721664373, 0.504297994269341], [-...",2,Camberwell,207011149,Whitehorse
1,4,2,3,NaN,"1 NEVIS STREET, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Nevis Street, Camberwell, Melbourne, City of B...",-37.843101,145.080403,850.0,2022-08-01,"[[-0.08487511721664373, -0.05063291139240506],...",2,Camberwell,207011149,Whitehorse
2,2,1,1,unknown,"1/10 GLENCAIRN AVENUE, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glencairn Avenue, Camberwell, Melbourne, City ...",-37.848191,145.080877,395.0,2021-05-01,"[[-7.915289157203776, 0.06756756756756757]]",1,Camberwell,207011149,Whitehorse
3,3,2,2,Townhouse,"1/1017 TOORAK ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Toorak Road, Camberwell, Melbourne, City of Bo...",-37.850405,145.095567,580.0,2021-07-01,"[[-2.4120960731568752, 0.008695652173913044], ...",5,Camberwell,207011149,Whitehorse
4,2,1,1,AUF,"1/11 EDDY STREET, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Eddy Street, Camberwell, Melbourne, City of Bo...",-37.846792,145.072722,400.0,2022-06-01,"[[-5.497717270032924, 0.02564102564102564], [-...",3,Camberwell,207011149,Whitehorse


In [97]:
len(df3)

69595

In [98]:
# metro_melb is a list of all LGAs of metropolitan melbourne
# that is defined https://liveinmelbourne.vic.gov.au/discover/melbourne-victoria/metropolitan-melbourne
metro_melb = ['Banyule', 'Bayside', 'Boroondara', 'Brimbank', 'Cardinia', 'Casey', 'Darebin', 'Frankston', 'Glen Eira', 'Greater Dandenong', 'Hobsons Bay', 'Hume', 'Kingston', 'Knox', 'Manningham', 'Maribyrnong', 'Maroondah', 'Melbourne', 'Melton', 'Monash', 'Moonee Valley', 'Moreland', 'Mornington Peninsula', 'Nillumbik', 'Port Phillip', 'Stonnington', 'Whitehorse', 'Whittlesea', 'Wyndham', 'Yarra', 'Yarra Ranges']

In [99]:
# only retain entries that are in metro melbourne as they are more related with the study
df3 = df3.loc[df3['lgaregion'].isin(metro_melb)]

In [100]:
# check the number of unique properties again
len(df3)

57778

### Embed income data

In [101]:
# the csv file was exported from MS Excel as Python's ability to read csv with weird formating is limited
income_df = pd.read_csv(f'{files_dire}/abs/income_distribution.csv')
# only retain information that will be used
income_df = income_df[['SA2', 'Earners', 'Median age of earners', 'Median', 'Mean', 'Top 10%']]
income_df = income_df.replace(',','', regex=True)
cols = ['Earners', 'Median age of earners', 'Median', 'Mean', 'Top 10%']
income_df[cols] = income_df[cols].apply(pd.to_numeric, errors='coerce', axis=1)
income_df = income_df.rename(columns={'Earners': 'income_earner', 'Median age of earners': 'income_median_age', 'Median': 'income_median', 'Mean': 'income_mean', 'Top 10%': 'income_top_10_pct'}, errors="raise")
# information will be 'vlookup'ed by SA2 code, the data contains the number of earners in the SA2, median age of the earner
# median income of the SA2, average income of the SA2 and how much of the population are ranked 10 percent within whole Australia
income_df.head()

,SA2,income_earner,income_median_age,income_median,income_mean,income_top_10_pct
0,201011001,7989.0,42.0,53932.0,63668.0,28.7
1,201011002,7595.0,47.0,53688.0,77876.0,38.4
2,201011003,13598.0,43.0,50593.0,60367.0,29.3
3,201011004,12722.0,40.0,45828.0,50778.0,26.1
4,201011005,4249.0,46.0,52377.0,63258.0,29.3


In [102]:
# merge income information
df3 = pd.merge(df3, income_df, on='SA2')
df3.head()

,bed,bath,car,type,address,suburb,postcode,url,loc_address,lat,...,list_history,list_count,SA2_NAME_2016,SA2,lgaregion,income_earner,income_median_age,income_median,income_mean,income_top_10_pct
0,3,1,2,AUF,". GLYNDON ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glyndon Road, Camberwell, Melbourne, City of B...",-37.836230,...,"[[-0.08487511721664373, 0.504297994269341], [-...",2,Camberwell,207011149,Whitehorse,14501.0,47.0,61520.0,101491.0,42.5
1,4,2,3,NaN,"1 NEVIS STREET, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Nevis Street, Camberwell, Melbourne, City of B...",-37.843101,...,"[[-0.08487511721664373, -0.05063291139240506],...",2,Camberwell,207011149,Whitehorse,14501.0,47.0,61520.0,101491.0,42.5
2,2,1,1,unknown,"1/10 GLENCAIRN AVENUE, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glencairn Avenue, Camberwell, Melbourne, City ...",-37.848191,...,"[[-7.915289157203776, 0.06756756756756757]]",1,Camberwell,207011149,Whitehorse,14501.0,47.0,61520.0,101491.0,42.5
3,3,2,2,Townhouse,"1/1017 TOORAK ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Toorak Road, Camberwell, Melbourne, City of Bo...",-37.850405,...,"[[-2.4120960731568752, 0.008695652173913044], ...",5,Camberwell,207011149,Whitehorse,14501.0,47.0,61520.0,101491.0,42.5
4,2,1,1,AUF,"1/11 EDDY STREET, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Eddy Street, Camberwell, Melbourne, City of Bo...",-37.846792,...,"[[-5.497717270032924, 0.02564102564102564], [-...",3,Camberwell,207011149,Whitehorse,14501.0,47.0,61520.0,101491.0,42.5


In [103]:
df3.to_csv('temp.csv')


In [104]:
len(df3)

57778

### Embed population statistics

In [105]:
# the csv file was exported from MS Excel as Python's ability to read csv with weird formating is limited
population_df = pd.read_csv(f'{files_dire}/abs/population.csv')
population_df = population_df.iloc[: , 8:]
# age are grouped into following categories
# they are sumed to produce these columns
# 0-14 years (children), 15-24 years (early working age), 25-54 years (prime working age), 55-64 years (mature working age), 65 years and over (elderly)
population_df['population_children'] = population_df[['0-4', '5–9', '10–14']].sum(axis = 1)
population_df = population_df.drop(['0-4', '5–9', '10–14'], axis=1)
population_df['population_prime_working'] = population_df[['25–29', '30–34', '35–39', '40–44', '45–49', '50–54']].sum(axis = 1)
population_df = population_df.drop(['25–29', '30–34', '35–39', '40–44', '45–49', '50–54'], axis=1)
population_df['population_elderly'] = population_df[['65–69', '70–74', '75–79', '80–84', '85 and over']].sum(axis = 1)
population_df = population_df.drop(['65–69', '70–74', '75–79', '80–84', '85 and over'], axis=1)
population_df['population_early_working'] = population_df.iloc[:,2] + population_df.iloc[:,3]
population_df['population_mature_working'] = population_df.iloc[:,4] + population_df.iloc[:,5]
population_df = population_df.drop(['15–19', '20–24', '55–59', '60–64', 'SA2 name'], axis=1)
population_df = population_df.rename(columns={'Total persons': 'population_total', 'SA2 code': 'SA2'}, errors="raise")
population_df.head()

,SA2,population_total,population_children,population_prime_working,population_elderly,population_early_working,population_mature_working
0,201011001,16823,4075,6702,2098,2391,1557
1,201011002,12076,1806,4321,2750,1565,1634
2,201011005,7232,1532,2483,1295,944,978
3,201011006,10640,2299,4347,1543,1434,1017
4,201011007,4213,904,1533,594,559,623


In [106]:
df3 = pd.merge(df3, population_df, on='SA2')
df3.head()

,bed,bath,car,type,address,suburb,postcode,url,loc_address,lat,...,income_median_age,income_median,income_mean,income_top_10_pct,population_total,population_children,population_prime_working,population_elderly,population_early_working,population_mature_working
0,3,1,2,AUF,". GLYNDON ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glyndon Road, Camberwell, Melbourne, City of B...",-37.836230,...,47.0,61520.0,101491.0,42.5,21512,3506,7721,4504,3069,2712
1,4,2,3,NaN,"1 NEVIS STREET, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Nevis Street, Camberwell, Melbourne, City of B...",-37.843101,...,47.0,61520.0,101491.0,42.5,21512,3506,7721,4504,3069,2712
2,2,1,1,unknown,"1/10 GLENCAIRN AVENUE, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glencairn Avenue, Camberwell, Melbourne, City ...",-37.848191,...,47.0,61520.0,101491.0,42.5,21512,3506,7721,4504,3069,2712
3,3,2,2,Townhouse,"1/1017 TOORAK ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Toorak Road, Camberwell, Melbourne, City of Bo...",-37.850405,...,47.0,61520.0,101491.0,42.5,21512,3506,7721,4504,3069,2712
4,2,1,1,AUF,"1/11 EDDY STREET, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Eddy Street, Camberwell, Melbourne, City of Bo...",-37.846792,...,47.0,61520.0,101491.0,42.5,21512,3506,7721,4504,3069,2712


In [107]:
len(df3)

42032

### Embed school locations

In [108]:
school_df = pd.read_csv('../data/raw/schools.csv')
school_df = school_df.drop(['Entity_Type', 'School_No', 'School_Status', 'Address_Line_1', 'Address_Line_2',
         'Address_Town', 'Address_State', 'Address_Postcode',
         'Postal_Address_Line_1', 'Postal_Address_Line_2', 'Postal_Town',
         'Postal_State', 'Postal_Postcode', 'Full_Phone_No', 'LGA_ID',
         'LGA_Name'], axis=1)
school_df['geo_coordinate'] = [(x, y) for x,y in zip(school_df['Y'], school_df['X'])]
# only retain public schools
school_df = school_df.loc[school_df['Education_Sector']=='Government']

In [109]:
# this is adapted from stackoverflow
# https://codereview.stackexchange.com/questions/28207/finding-the-closest-point-to-a-list-of-points
def closest_point(point, points):
    # Find closest point from a list of points.
    return points[cdist([point], points).argmin()]
def match_value(df, col1, x, col2):
    # Match value x from col1 row to value in col2.
    return df[df[col1] == x][col2].values[0]

In [110]:
df3['geo_coordinate'] = [(x, y) for x,y in zip(df3['lat'], df3['lon'])]
df3.head()

,bed,bath,car,type,address,suburb,postcode,url,loc_address,lat,...,income_median,income_mean,income_top_10_pct,population_total,population_children,population_prime_working,population_elderly,population_early_working,population_mature_working,geo_coordinate
0,3,1,2,AUF,". GLYNDON ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glyndon Road, Camberwell, Melbourne, City of B...",-37.836230,...,61520.0,101491.0,42.5,21512,3506,7721,4504,3069,2712,"(-37.8362297, 145.0790033)"
1,4,2,3,NaN,"1 NEVIS STREET, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Nevis Street, Camberwell, Melbourne, City of B...",-37.843101,...,61520.0,101491.0,42.5,21512,3506,7721,4504,3069,2712,"(-37.8431012, 145.080403)"
2,2,1,1,unknown,"1/10 GLENCAIRN AVENUE, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glencairn Avenue, Camberwell, Melbourne, City ...",-37.848191,...,61520.0,101491.0,42.5,21512,3506,7721,4504,3069,2712,"(-37.8481908, 145.0808774)"
3,3,2,2,Townhouse,"1/1017 TOORAK ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Toorak Road, Camberwell, Melbourne, City of Bo...",-37.850405,...,61520.0,101491.0,42.5,21512,3506,7721,4504,3069,2712,"(-37.8504051, 145.0955675)"
4,2,1,1,AUF,"1/11 EDDY STREET, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Eddy Street, Camberwell, Melbourne, City of Bo...",-37.846792,...,61520.0,101491.0,42.5,21512,3506,7721,4504,3069,2712,"(-37.8467915, 145.0727223)"


##### primary schools

In [111]:
school_type = ['Primary', 'Pri/Sec']
school_primary_df = school_df.loc[school_df['School_Type'].isin(school_type)]

In [112]:
# primary school
df3['closest_primary_school_loc'] = tqdm([closest_point(x, list(school_primary_df['geo_coordinate'])) for x in df3['geo_coordinate']])
df3['primary_school_name'] = tqdm([match_value(school_primary_df, 'geo_coordinate', x, 'School_Name') for x in df3['closest_primary_school_loc']])

100%|██████████| 42032/42032 [00:00<00:00, 5164337.40it/s]


##### secondary schools

In [113]:
school_type = ['Secondary', 'Pri/Sec']
school_secondary_df = school_df.loc[school_df['School_Type'].isin(school_type)]

In [114]:
# embed secondary school ranking information
# considering some family may rent a property for its free education
school_ranking = pd.read_html('https://bettereducation.com.au/Results/vcePublicSchoolResults.aspx')[-1]
school_ranking = school_ranking[['Better Education Rank', 'School', 'Unit 3-4 cohort']]
school_ranking = school_ranking.rename(columns={'School': 'School_Name', 'Better Education Rank': 'secondary_school_rank', 'Unit 3-4 cohort': 'secondary_cohort'}, errors="coerce")
school_secondary_df = pd.merge(school_secondary_df, school_ranking, on='School_Name')

In [115]:
df3['closest_secondary_school_loc'] = tqdm([closest_point(x, list(school_secondary_df['geo_coordinate'])) for x in df3['geo_coordinate']])
df3['secondary_school_name'] = tqdm([match_value(school_secondary_df, 'geo_coordinate', x, 'School_Name') for x in df3['closest_secondary_school_loc']])
df3['secondary_school_rank'] = tqdm([match_value(school_secondary_df, 'geo_coordinate', x, 'secondary_school_rank') for x in df3['closest_secondary_school_loc']])
df3['secondary_school_cohort'] = tqdm([match_value(school_secondary_df, 'geo_coordinate', x, 'secondary_cohort') for x in df3['closest_secondary_school_loc']])

100%|██████████| 42032/42032 [00:00<00:00, 4800408.05it/s]


In [116]:
df3['closest_primary_school_loc'] = df3['closest_primary_school_loc'].astype('str')
df3['closest_secondary_school_loc'] = df3['closest_secondary_school_loc'].astype('str')

In [117]:
df3[['pri_lat', 'pri_lon']] = df3['closest_primary_school_loc'].str.split(', ', 1, expand=True)
df3[['sec_lat', 'sec_lon']] = df3['closest_secondary_school_loc'].str.split(',', 1, expand=True)

/var/folders/y5/myhyydfd6h9dlwv1qmqtqhmr0000gn/T/ipykernel_54504/1371061774.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df3[['pri_lat', 'pri_lon']] = df3['closest_primary_school_loc'].str.split(', ', 1, expand=True)
/var/folders/y5/myhyydfd6h9dlwv1qmqtqhmr0000gn/T/ipykernel_54504/1371061774.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df3[['sec_lat', 'sec_lon']] = df3['closest_secondary_school_loc'].str.split(',', 1, expand=True)


In [118]:
bracket_remove = ['pri_lat', 'pri_lon', 'sec_lat', 'sec_lon']
for i in bracket_remove:
    df3[i] = df3[i].str.strip('()')
df3[bracket_remove] = df3[bracket_remove].apply(pd.to_numeric, errors='coerce')

In [119]:
df3 = df3.drop(['closest_primary_school_loc', 'closest_secondary_school_loc'], axis = 1)

In [120]:
df3

,bed,bath,car,type,address,suburb,postcode,url,loc_address,lat,...,population_mature_working,geo_coordinate,primary_school_name,secondary_school_name,secondary_school_rank,secondary_school_cohort,pri_lat,pri_lon,sec_lat,sec_lon
0,3,1,2,AUF,". GLYNDON ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glyndon Road, Camberwell, Melbourne, City of B...",-37.836230,...,2712,"(-37.8362297, 145.0790033)",Canterbury Primary School,Camberwell High School,40,290,-37.830963,145.083144,-37.830151,145.072821
1,4,2,3,NaN,"1 NEVIS STREET, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Nevis Street, Camberwell, Melbourne, City of B...",-37.843101,...,2712,"(-37.8431012, 145.080403)",Hartwell Primary School,Camberwell High School,40,290,-37.848051,145.084466,-37.830151,145.072821
2,2,1,1,unknown,"1/10 GLENCAIRN AVENUE, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glencairn Avenue, Camberwell, Melbourne, City ...",-37.848191,...,2712,"(-37.8481908, 145.0808774)",Hartwell Primary School,Camberwell High School,40,290,-37.848051,145.084466,-37.830151,145.072821
3,3,2,2,Townhouse,"1/1017 TOORAK ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Toorak Road, Camberwell, Melbourne, City of Bo...",-37.850405,...,2712,"(-37.8504051, 145.0955675)",Wattle Park Primary School,Ashwood High School,17,42,-37.842635,145.097370,-37.864527,145.103328
4,2,1,1,AUF,"1/11 EDDY STREET, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Eddy Street, Camberwell, Melbourne, City of Bo...",-37.846792,...,2712,"(-37.8467915, 145.0727223)",Camberwell South Primary School,Camberwell High School,40,290,-37.847905,145.062612,-37.830151,145.072821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42027,3,1,1,House,"1486 BOLINDA ROAD, DARRAWEIT GUIM",Darraweit Guim,3756,https://www.oldlistings.com.au/real-estate/VIC...,"Bolinda - Darraweit Road, Bolinda, Shire of Ma...",-37.438871,...,1509,"(-37.4388711, 144.8236433)",Bolinda Primary School,Gisborne Secondary College,80,212,-37.433735,144.745404,-37.498114,144.584774
42028,3,1,2,unknown,"2264 WHITTLESEA-YEA ROAD, HAZELDENE",Hazeldene,3658,https://www.oldlistings.com.au/real-estate/VIC...,"Whittlesea - Yea Road, Flowerdale, Hazeldene, ...",-37.342711,...,787,"(-37.3427105, 145.2889966)",Flowerdale Primary School,Mernda Central P-12 College,68,26,-37.316923,145.291361,-37.604853,145.088330
42029,3,2,4,unknown,"3041 BROADFORD-FLOWERDALE ROAD, FLOWERDALE",Flowerdale,3658,https://www.oldlistings.com.au/real-estate/VIC...,"Broadford - Flowerdale Road, Flowerdale, Shire...",-37.312564,...,787,"(-37.3125639, 145.2926123)",Flowerdale Primary School,Mernda Central P-12 College,68,26,-37.316923,145.291361,-37.604853,145.088330
42030,5,2,3,unknown,"235 PARWAN EXFORD ROAD, PARWAN",Parwan,3340,https://www.oldlistings.com.au/real-estate/VIC...,"Parwan Exford Road, Parwan, Melbourne, City of...",-37.725486,...,958,"(-37.7254861, 144.5061106)",Exford Primary School,Gisborne Secondary College,80,212,-37.745287,144.558791,-37.498114,144.584774


In [121]:
df3.dtypes

bed                                   int64
bath                                  int64
car                                   int64
type                                 object
address                              object
suburb                               object
postcode                              int64
url                                  object
loc_address                          object
lat                                 float64
lon                                 float64
weekly_rent                         float64
list_date                    datetime64[ns]
list_history                         object
list_count                            int64
SA2_NAME_2016                        object
SA2                                   int64
lgaregion                            object
income_earner                       float64
income_median_age                   float64
income_median                       float64
income_mean                         float64
income_top_10_pct               

### embed population projection

In [122]:
population_projection_df = pd.read_csv('../data/raw/population_projection.csv')
population_projection_df = population_projection_df.loc[population_projection_df['SEX'] == 'Persons']
population_projection_df = population_projection_df.loc[population_projection_df['YEAR'] == 2027]
population_projection_df = population_projection_df.drop(['YEAR', 'SA2_NAME', 'SEX'], axis=1)
# they are categories as well with the previous population statistics
# 0-14 years (children), 15-24 years (early working age), 25-54 years (prime working age), 55-64 years (mature working age), 65 years and over (elderly)
population_projection_df['proj_population_children'] = population_projection_df[['Age0-4', 'Age5-9', 'Age10-14']].sum(axis = 1)
population_projection_df = population_projection_df.drop(['Age0-4', 'Age5-9', 'Age10-14'], axis=1)
population_projection_df['proj_population_early_working'] = population_projection_df[['Age15-19', 'Age20-24']].sum(axis = 1)
population_projection_df = population_projection_df.drop(['Age15-19', 'Age20-24'], axis=1)
population_projection_df['proj_population_prime_working'] = population_projection_df[['Age25-29', 'Age30-34', 'Age35-39', 'Age40-44', 'Age45-49', 'Age50-54']].sum(axis = 1)
population_projection_df = population_projection_df.drop(['Age25-29', 'Age30-34', 'Age35-39', 'Age40-44', 'Age45-49', 'Age50-54'], axis=1)
population_projection_df['proj_population_mature_working'] = population_projection_df[['Age55-59', 'Age60-64']].sum(axis = 1)
population_projection_df = population_projection_df.drop(['Age55-59', 'Age60-64'], axis=1)
population_projection_df['proj_population_elderly'] = population_projection_df[['Age65-69', 'Age70-74', 'Age75-79', 'Age80-84', 'Age85+']].sum(axis = 1)
population_projection_df = population_projection_df.drop(['Age65-69', 'Age70-74', 'Age75-79', 'Age80-84', 'Age85+'], axis=1)
population_projection_df = population_projection_df.rename(columns={'Total': 'proj_population_total', 'SA2_CODE': 'SA2'}, errors="raise")
population_projection_df.head()

,SA2,proj_population_total,proj_population_children,proj_population_early_working,proj_population_prime_working,proj_population_mature_working,proj_population_elderly
13862,201011001,18611,4389,2496,7275,1807,2644
13865,201011002,12252,1788,1796,3940,1674,3054
13868,201011003,26630,5094,2777,9809,3270,5680
13871,201011004,28423,4849,3734,11347,3056,5437
13874,201011005,8900,1919,1368,2966,1067,1580


In [123]:
df3 = pd.merge(df3, population_projection_df, on='SA2')
df3.head()

,bed,bath,car,type,address,suburb,postcode,url,loc_address,lat,...,pri_lat,pri_lon,sec_lat,sec_lon,proj_population_total,proj_population_children,proj_population_early_working,proj_population_prime_working,proj_population_mature_working,proj_population_elderly
0,3,1,2,AUF,". GLYNDON ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glyndon Road, Camberwell, Melbourne, City of B...",-37.836230,...,-37.830963,145.083144,-37.830151,145.072821,25643,4215,4019,9588,2937,4884
1,4,2,3,NaN,"1 NEVIS STREET, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Nevis Street, Camberwell, Melbourne, City of B...",-37.843101,...,-37.848051,145.084466,-37.830151,145.072821,25643,4215,4019,9588,2937,4884
2,2,1,1,unknown,"1/10 GLENCAIRN AVENUE, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glencairn Avenue, Camberwell, Melbourne, City ...",-37.848191,...,-37.848051,145.084466,-37.830151,145.072821,25643,4215,4019,9588,2937,4884
3,3,2,2,Townhouse,"1/1017 TOORAK ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Toorak Road, Camberwell, Melbourne, City of Bo...",-37.850405,...,-37.842635,145.097370,-37.864527,145.103328,25643,4215,4019,9588,2937,4884
4,2,1,1,AUF,"1/11 EDDY STREET, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Eddy Street, Camberwell, Melbourne, City of Bo...",-37.846792,...,-37.847905,145.062612,-37.830151,145.072821,25643,4215,4019,9588,2937,4884


In [124]:
len(df3)

42032

### hospitals

In [125]:
hospitals_df = pd.read_csv('../data/raw/hospitals.csv')
hospitals_df = hospitals_df.loc[hospitals_df['Emergency Capable'] == 'YES']
hospitals_df['full_address'] = hospitals_df['Location Address'] + ', ' +hospitals_df['Suburb'] + ', Victoria'
hospitals_df = hospitals_df[['Formal Name', 'full_address']]

In [126]:
# get the geocode of each hospital
temp = []
for i in tqdm(hospitals_df['full_address']):
    locator = geopy.Nominatim(user_agent="myGeocoder");
    location = locator.geocode(i,timeout=None);
    if location != None:
        info = [i, location.latitude, location.longitude]
        temp.append(info)

100%|██████████| 39/39 [00:33<00:00,  1.18it/s]


In [127]:
hospital_geo = pd.DataFrame(temp, columns=['full_address', 'lat', 'lon'])
hospitals_df = pd.merge(hospitals_df, hospital_geo, on=['full_address'])
hospitals_df = hospitals_df.drop('full_address', axis=1)
hospitals_df['geo_coordinate'] = [(x, y) for x,y in zip(hospitals_df['lat'], hospitals_df['lon'])]

In [128]:
hospitals_df.head()

,Formal Name,lat,lon,geo_coordinate
0,Albury Wodonga Health,-36.132040,146.880004,"(-36.1320403, 146.8800037)"
1,Angliss Hospital,-37.896595,145.310108,"(-37.8965948, 145.3101077)"
2,Austin Health - Austin Hospital,-37.756561,145.058657,"(-37.75656125, 145.05865657809136)"
3,Bairnsdale Regional Health Service,-37.817113,147.611512,"(-37.8171132, 147.6115117)"
4,Bendigo Health,-36.748620,144.285198,"(-36.7486197, 144.2851975)"


In [129]:
df3['closest_ed_loc'] = tqdm([closest_point(x, list(hospitals_df['geo_coordinate'])) for x in df3['geo_coordinate']])
df3['closest_ed_name'] = tqdm([match_value(hospitals_df, 'geo_coordinate', x, 'Formal Name') for x in df3['closest_ed_loc']])

100%|██████████| 42032/42032 [00:00<00:00, 5378454.63it/s]


In [130]:
df3['closest_ed_loc'] = df3['closest_ed_loc'].astype('str')
df3[['ed_lat', 'ed_lon']] = df3['closest_ed_loc'].str.split(', ', 1, expand=True)

/var/folders/y5/myhyydfd6h9dlwv1qmqtqhmr0000gn/T/ipykernel_54504/2338036252.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df3[['ed_lat', 'ed_lon']] = df3['closest_ed_loc'].str.split(', ', 1, expand=True)


In [131]:
bracket_remove = ['ed_lat', 'ed_lon']
for i in bracket_remove:
    df3[i] = df3[i].str.strip('()')
df3[bracket_remove] = df3[bracket_remove].apply(pd.to_numeric, errors='coerce')

In [132]:
df3 = df3.drop('closest_ed_loc', axis = 1)

In [133]:
df3

,bed,bath,car,type,address,suburb,postcode,url,loc_address,lat,...,sec_lon,proj_population_total,proj_population_children,proj_population_early_working,proj_population_prime_working,proj_population_mature_working,proj_population_elderly,closest_ed_name,ed_lat,ed_lon
0,3,1,2,AUF,". GLYNDON ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glyndon Road, Camberwell, Melbourne, City of B...",-37.836230,...,145.072821,25643,4215,4019,9588,2937,4884,Box Hill Hospital,-37.815458,145.119672
1,4,2,3,NaN,"1 NEVIS STREET, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Nevis Street, Camberwell, Melbourne, City of B...",-37.843101,...,145.072821,25643,4215,4019,9588,2937,4884,Box Hill Hospital,-37.815458,145.119672
2,2,1,1,unknown,"1/10 GLENCAIRN AVENUE, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glencairn Avenue, Camberwell, Melbourne, City ...",-37.848191,...,145.072821,25643,4215,4019,9588,2937,4884,Box Hill Hospital,-37.815458,145.119672
3,3,2,2,Townhouse,"1/1017 TOORAK ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Toorak Road, Camberwell, Melbourne, City of Bo...",-37.850405,...,145.103328,25643,4215,4019,9588,2937,4884,Box Hill Hospital,-37.815458,145.119672
4,2,1,1,AUF,"1/11 EDDY STREET, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Eddy Street, Camberwell, Melbourne, City of Bo...",-37.846792,...,145.072821,25643,4215,4019,9588,2937,4884,Box Hill Hospital,-37.815458,145.119672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42027,3,1,1,House,"1486 BOLINDA ROAD, DARRAWEIT GUIM",Darraweit Guim,3756,https://www.oldlistings.com.au/real-estate/VIC...,"Bolinda - Darraweit Road, Bolinda, Shire of Ma...",-37.438871,...,144.584774,11968,2520,1436,4316,1581,2115,The Northern Hospital,-37.653398,145.014504
42028,3,1,2,unknown,"2264 WHITTLESEA-YEA ROAD, HAZELDENE",Hazeldene,3658,https://www.oldlistings.com.au/real-estate/VIC...,"Whittlesea - Yea Road, Flowerdale, Hazeldene, ...",-37.342711,...,145.088330,3968,592,422,1050,551,1353,The Northern Hospital,-37.653398,145.014504
42029,3,2,4,unknown,"3041 BROADFORD-FLOWERDALE ROAD, FLOWERDALE",Flowerdale,3658,https://www.oldlistings.com.au/real-estate/VIC...,"Broadford - Flowerdale Road, Flowerdale, Shire...",-37.312564,...,145.088330,3968,592,422,1050,551,1353,The Northern Hospital,-37.653398,145.014504
42030,5,2,3,unknown,"235 PARWAN EXFORD ROAD, PARWAN",Parwan,3340,https://www.oldlistings.com.au/real-estate/VIC...,"Parwan Exford Road, Parwan, Melbourne, City of...",-37.725486,...,144.584774,6584,1190,599,2157,943,1695,Werribee Mercy Hospital,-37.899809,144.667923


#### train station

In [134]:
train_df = gpd.read_file('../data/raw/PTV/PTV_METRO_TRAIN_STATION.shp')
train_df = train_df[['STOP_NAME', 'LATITUDE', 'LONGITUDE', 'ROUTEUSSP']]
train_df['geo_coordinate'] = [(x, y) for x, y in zip(train_df['LATITUDE'], train_df['LONGITUDE'])]
train_df['n_routes'] = train_df['ROUTEUSSP'].str.count(',') + 1
train_df = train_df.drop(['LONGITUDE', 'LATITUDE', 'ROUTEUSSP'], axis=1)
train_df

,STOP_NAME,geo_coordinate,n_routes
0,Royal Park Railway Station (Parkville),"(-37.781193, 144.952301)",1
1,Flemington Bridge Railway Station (North Melbo...,"(-37.78814, 144.939323)",1
2,Macaulay Railway Station (North Melbourne),"(-37.794267, 144.936166)",1
3,North Melbourne Railway Station (West Melbourne),"(-37.807419, 144.94257)",6
4,Clifton Hill Railway Station (Clifton Hill),"(-37.788657, 144.995417)",2
...,...,...,...
215,Coburg Railway Station (Coburg),"(-37.742345, 144.963336)",1
216,Moreland Railway Station (Coburg),"(-37.754485, 144.961823)",1
217,Anstey Railway Station (Brunswick),"(-37.761242, 144.960684)",1
218,Brunswick Railway Station (Brunswick),"(-37.767721, 144.959587)",1


In [135]:
df3['closest_train_loc'] = tqdm([closest_point(x, list(train_df['geo_coordinate'])) for x in df3['geo_coordinate']])
df3['train_stop'] = tqdm([match_value(train_df, 'geo_coordinate', x, 'STOP_NAME') for x in df3['closest_train_loc']])
df3['train_n_lines'] = tqdm([match_value(train_df, 'geo_coordinate', x, 'n_routes') for x in df3['closest_train_loc']])

100%|██████████| 42032/42032 [00:00<00:00, 5190948.29it/s]


In [136]:
df3['closest_train_loc'] = df3['closest_train_loc'].astype('str')
df3[['train_lat', 'train_lon']] = df3['closest_train_loc'].str.split(', ', 1, expand=True)

/var/folders/y5/myhyydfd6h9dlwv1qmqtqhmr0000gn/T/ipykernel_54504/284886828.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df3[['train_lat', 'train_lon']] = df3['closest_train_loc'].str.split(', ', 1, expand=True)


In [137]:
bracket_remove = ['train_lat', 'train_lon']
for i in bracket_remove:
    df3[i] = df3[i].str.strip('()')
df3[bracket_remove] = df3[bracket_remove].apply(pd.to_numeric, errors='coerce')
df3 = df3.drop('closest_train_loc', axis=1)

In [138]:
df3

,bed,bath,car,type,address,suburb,postcode,url,loc_address,lat,...,proj_population_prime_working,proj_population_mature_working,proj_population_elderly,closest_ed_name,ed_lat,ed_lon,train_stop,train_n_lines,train_lat,train_lon
0,3,1,2,AUF,". GLYNDON ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glyndon Road, Camberwell, Melbourne, City of B...",-37.836230,...,9588,2937,4884,Box Hill Hospital,-37.815458,145.119672,Hartwell Railway Station (Camberwell),1,-37.843985,145.075560
1,4,2,3,NaN,"1 NEVIS STREET, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Nevis Street, Camberwell, Melbourne, City of B...",-37.843101,...,9588,2937,4884,Box Hill Hospital,-37.815458,145.119672,Hartwell Railway Station (Camberwell),1,-37.843985,145.075560
2,2,1,1,unknown,"1/10 GLENCAIRN AVENUE, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Glencairn Avenue, Camberwell, Melbourne, City ...",-37.848191,...,9588,2937,4884,Box Hill Hospital,-37.815458,145.119672,Burwood Railway Station (Glen Iris),1,-37.851563,145.080511
3,3,2,2,Townhouse,"1/1017 TOORAK ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Toorak Road, Camberwell, Melbourne, City of Bo...",-37.850405,...,9588,2937,4884,Box Hill Hospital,-37.815458,145.119672,Burwood Railway Station (Glen Iris),1,-37.851563,145.080511
4,2,1,1,AUF,"1/11 EDDY STREET, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Eddy Street, Camberwell, Melbourne, City of Bo...",-37.846792,...,9588,2937,4884,Box Hill Hospital,-37.815458,145.119672,Hartwell Railway Station (Camberwell),1,-37.843985,145.075560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42027,3,1,1,House,"1486 BOLINDA ROAD, DARRAWEIT GUIM",Darraweit Guim,3756,https://www.oldlistings.com.au/real-estate/VIC...,"Bolinda - Darraweit Road, Bolinda, Shire of Ma...",-37.438871,...,4316,1581,2115,The Northern Hospital,-37.653398,145.014504,Sunbury Railway Station (Sunbury),1,-37.579091,144.727319
42028,3,1,2,unknown,"2264 WHITTLESEA-YEA ROAD, HAZELDENE",Hazeldene,3658,https://www.oldlistings.com.au/real-estate/VIC...,"Whittlesea - Yea Road, Flowerdale, Hazeldene, ...",-37.342711,...,1050,551,1353,The Northern Hospital,-37.653398,145.014504,Hurstbridge Railway Station (Hurstbridge),1,-37.639398,145.192017
42029,3,2,4,unknown,"3041 BROADFORD-FLOWERDALE ROAD, FLOWERDALE",Flowerdale,3658,https://www.oldlistings.com.au/real-estate/VIC...,"Broadford - Flowerdale Road, Flowerdale, Shire...",-37.312564,...,1050,551,1353,The Northern Hospital,-37.653398,145.014504,Hurstbridge Railway Station (Hurstbridge),1,-37.639398,145.192017
42030,5,2,3,unknown,"235 PARWAN EXFORD ROAD, PARWAN",Parwan,3340,https://www.oldlistings.com.au/real-estate/VIC...,"Parwan Exford Road, Parwan, Melbourne, City of...",-37.725486,...,2157,943,1695,Werribee Mercy Hospital,-37.899809,144.667923,Werribee Railway Station (Werribee),1,-37.899378,144.661118


### final check and export

In [139]:
df3 = df3.drop('geo_coordinate', axis = 1)

In [140]:
df3.isna().sum()
# consider we still have a lot of entries
# and we want to produce the best model
# drop all rows with any NaN
# this shall not be applied if the dataset is small

bed                                  0
bath                                 0
car                                  0
type                              6997
address                              0
suburb                               0
postcode                             0
url                                  0
loc_address                          0
lat                                  0
lon                                  0
weekly_rent                          0
list_date                            0
list_history                         0
list_count                           0
SA2_NAME_2016                        0
SA2                                  0
lgaregion                            0
income_earner                        0
income_median_age                    0
income_median                        0
income_mean                          0
income_top_10_pct                  232
population_total                     0
population_children                  0
population_prime_working 

In [141]:
df3['type'] =  df['type'].fillna('unknown')

In [142]:
df3[df3.isna().any(axis=1)]

,bed,bath,car,type,address,suburb,postcode,url,loc_address,lat,...,proj_population_prime_working,proj_population_mature_working,proj_population_elderly,closest_ed_name,ed_lat,ed_lon,train_stop,train_n_lines,train_lat,train_lon
8,3,2,1,NaN,"1/1306 TOORAK ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Toorak Road, Camberwell, Melbourne, City of Bo...",-37.850405,...,9588,2937,4884,Box Hill Hospital,-37.815458,145.119672,Burwood Railway Station (Glen Iris),1,-37.851563,145.080511
9,3,2,2,NaN,"1/1349 TOORAK ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Toorak Road, Camberwell, Melbourne, City of Bo...",-37.850405,...,9588,2937,4884,Box Hill Hospital,-37.815458,145.119672,Burwood Railway Station (Glen Iris),1,-37.851563,145.080511
10,2,1,1,NaN,"1/17 GARDEN ROAD, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Garden Road, Camberwell, Melbourne, City of Bo...",-37.843613,...,9588,2937,4884,Box Hill Hospital,-37.815458,145.119672,Burwood Railway Station (Glen Iris),1,-37.851563,145.080511
13,2,1,1,NaN,"1/25 THOMAS STREET, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Thomas Street, Camberwell, Melbourne, City of ...",-37.842711,...,9588,2937,4884,Box Hill Hospital,-37.815458,145.119672,Burwood Railway Station (Glen Iris),1,-37.851563,145.080511
14,3,2,2,NaN,"1/3 HALLEY AVENUE, CAMBERWELL",Camberwell,3124,https://www.oldlistings.com.au/real-estate/VIC...,"Halley Avenue, Camberwell, Melbourne, City of ...",-37.846338,...,9588,2937,4884,Box Hill Hospital,-37.815458,145.119672,Hartwell Railway Station (Camberwell),1,-37.843985,145.075560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42027,3,1,1,NaN,"1486 BOLINDA ROAD, DARRAWEIT GUIM",Darraweit Guim,3756,https://www.oldlistings.com.au/real-estate/VIC...,"Bolinda - Darraweit Road, Bolinda, Shire of Ma...",-37.438871,...,4316,1581,2115,The Northern Hospital,-37.653398,145.014504,Sunbury Railway Station (Sunbury),1,-37.579091,144.727319
42028,3,1,2,NaN,"2264 WHITTLESEA-YEA ROAD, HAZELDENE",Hazeldene,3658,https://www.oldlistings.com.au/real-estate/VIC...,"Whittlesea - Yea Road, Flowerdale, Hazeldene, ...",-37.342711,...,1050,551,1353,The Northern Hospital,-37.653398,145.014504,Hurstbridge Railway Station (Hurstbridge),1,-37.639398,145.192017
42029,3,2,4,NaN,"3041 BROADFORD-FLOWERDALE ROAD, FLOWERDALE",Flowerdale,3658,https://www.oldlistings.com.au/real-estate/VIC...,"Broadford - Flowerdale Road, Flowerdale, Shire...",-37.312564,...,1050,551,1353,The Northern Hospital,-37.653398,145.014504,Hurstbridge Railway Station (Hurstbridge),1,-37.639398,145.192017
42030,5,2,3,NaN,"235 PARWAN EXFORD ROAD, PARWAN",Parwan,3340,https://www.oldlistings.com.au/real-estate/VIC...,"Parwan Exford Road, Parwan, Melbourne, City of...",-37.725486,...,2157,943,1695,Werribee Mercy Hospital,-37.899809,144.667923,Werribee Railway Station (Werribee),1,-37.899378,144.661118


In [143]:
# df3 = df3.dropna()

In [144]:
len(df3)

42032

In [145]:
features = ['lat', 'lon', 'pri_lat', 'pri_lon', 'sec_lat', 'sec_lon', 'ed_lat', 'ed_lon', 'train_lat', 'train_lon']

In [146]:
df3 = df3[df3['lat'] < -34]
df3 = df3[df3['lat'] > -40]
df3 = df3[df3['lon'] > 140]
df3 = df3[df3['lon'] < 150]

In [147]:
len(df3)

41980

In [148]:
df3.to_csv('../data/curated/listing_with_features.csv')

In [149]:
for i in features:
    print(i)
    print(df3[i].min())
    print(df3[i].max())

lat
-38.4726409
-36.9350715
lon
144.324648
146.3497923
pri_lat
-38.41187
-36.755962
pri_lon
144.255106
146.25688
sec_lat
-38.367615
-37.062676
sec_lon
144.340792
146.088565
ed_lat
-38.3629259
-36.35411275
ed_lon
144.667923
146.31380413326383
train_lat
-38.374235
-37.579091
train_lon
144.661118
145.486379
